## Ray core operation examples

### Configurations

In [1]:
! pip install --user -q "google-cloud-aiplatform[ray]>=1.56.0" \
                        "ray[data,train,tune,serve]>=2.33.0"

In [2]:
# @title Define constants
PROJECT_NBR = "721521243942"
PROJECT_ID = "ai-hangsik"
REGION="us-central1"
RAY_CLUSTER_NM = "ray33-cluster-20250216-192557"

In [3]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [4]:
import ray
from ray.runtime_env import RuntimeEnv
from ray.air.config import RunConfig
from ray.air import CheckpointConfig, ScalingConfig
from ray.util.joblib import register_ray

In [5]:
ray.__version__

'2.33.0'

### Connect to Ray on Vertex AI

In [54]:

ray.shutdown()

In [45]:

RAY_ADDRESS=f"vertex_ray://projects/{PROJECT_NBR}/locations/{REGION}/persistentResources/{RAY_CLUSTER_NM}"
print(f"RAY_ADDRESS:{RAY_ADDRESS}")

RUNTIME_ENV = {
  "pip": [
      "google-cloud-aiplatform[ray]>=1.56.0",
      "ray[data,train,tune,serve]>=2.33.0",
      "datasets",
      "evaluate",
      "accelerate==0.18.0",
      "transformers==4.26.0",
      "torch>=1.12.0",
      "deepspeed==0.12.3",
  ],
}

ray.init(address=RAY_ADDRESS,runtime_env=RUNTIME_ENV)

RAY_ADDRESS:vertex_ray://projects/721521243942/locations/us-central1/persistentResources/cluster-20250216-114448
[Ray on Vertex AI]: Cluster State = State.RUNNING


Python version:,3.10.16
Ray version:,2.33.0
Vertex SDK version:,1.80.0
Dashboard:,13e5361e220617fd-dot-us-central1.aiplatform-training.googleusercontent.com
Interactive Terminal Uri:,f2a22d401ce07eb0-dot-us-central1.aiplatform-training.googleusercontent.com
Cluster Name:,cluster-20250216-114448


### Ray Basic concept

#### Basic Operation

In [9]:
import time

# Define the square task.
@ray.remote
def square(x):
    time.sleep(3)
    return x * x

# Launch four parallel square tasks.
futures = [square.remote(i) for i in range(3)]

# Retrieve results.
print(ray.get(futures))
# -> [0, 1, 4,]

[0, 1, 4]


#### Define actor and operate

In [10]:
# Define the Counter actor.
@ray.remote
class Counter:
    def __init__(self):
        self.i = 0

    def get(self):
        return self.i

    def increase(self, value):
        self.i += value

# Create a Counter actor.
c = Counter.remote()

# Submit calls to the actor. These calls run asynchronously but in
# submission order on the remote actor process.
for _ in range(10):
    c.increase.remote(1)

# Retrieve final actor state.
print(ray.get(c.get.remote()))
# -> 10

10


#### Passing an Object

In [13]:
import numpy as np

# Define a task that sums the values in a matrix.
@ray.remote
def sum_matrix(matrix):
    return np.sum(matrix)

# Call the task with a literal argument value.
print(ray.get(sum_matrix.remote(np.ones((100, 100)))))
# -> 10000.0

# Put a large array into the object store.
matrix_ref = ray.put(np.ones((1000, 1000)))

# Call the task with the object reference as an argument.
print(ray.get(sum_matrix.remote(matrix_ref)))
# -> 1000000.0

10000.0
1000000.0


### Tasks
* https://docs.ray.io/en/latest/ray-core/tasks.html

In [11]:
import ray
import time


# A regular Python function.
def normal_function():
    return 1


# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def my_function():
    return 1


# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process.
obj_ref = my_function.remote()

# The result can be retrieved with ``ray.get``.
assert ray.get(obj_ref) == 1

In [20]:
@ray.remote
def slow_function():
    time.sleep(10)
    return 1


# Ray tasks are executed in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
for _ in range(4):
    # This doesn't block.
    slow_function.remote()

#### Specifying required resources

In [12]:
# Specify required resources.
@ray.remote(num_cpus=4, num_gpus=2)
def my_function():
    return 1

# Override the default resource requirements.
my_function.options(num_cpus=3).remote()

ClientObjectRef(1e360ffa862f8fe3ffffffffffffffffffffffff0100000001000000)

#### Passing object refs to Ray taks

* myfunction --> refs --> function with an argument --> refs.

In [13]:
@ray.remote
def my_function():
    return 1

@ray.remote
def function_with_an_argument(value):
    return value + 1

obj_ref1 = my_function.remote()
result1 = ray.get(obj_ref1)
print(result1)

# You can pass an object ref as an argument to another Ray task.
obj_ref2 = function_with_an_argument.remote(obj_ref1)
result2 = ray.get(obj_ref2)
print(result2)

1
2


#### wait for the slow function.

In [15]:
@ray.remote
def slow_function():
    time.sleep(10)
    return 1


object_refs = [slow_function.remote() for _ in range(2)]
# Return as soon as one of the tasks finished execution.
ready_refs, remaining_refs = ray.wait(object_refs, num_returns=1, timeout=None)

print(f"ready_refs:{ready_refs}")
print(f"remaining_refs:{remaining_refs}")


ready_refs:[ClientObjectRef(c76a79b2875a7251ffffffffffffffffffffffff0100000001000000)]
remaining_refs:[ClientObjectRef(465c0fb8d6cb3cdcffffffffffffffffffffffff0100000001000000)]


#### Multiple resturns

In [16]:
# By default, a Ray task only returns a single Object Ref.
@ray.remote
def return_single():
    return 0, 1, 2  # tuple return.


object_ref = return_single.remote()
assert ray.get(object_ref) == (0, 1, 2)


# However, you can configure Ray tasks to return multiple Object Refs.
@ray.remote(num_returns=3)
def return_multiple():
    return 0, 1, 2


object_ref0, object_ref1, object_ref2 = return_multiple.remote()

ray.get(object_ref0), ray.get(object_ref1), ray.get(object_ref2)

(0, 1, 2)

### Actors

#### Actors basic

In [48]:
import ray

#@ray.remote(num_cpus=1, num_gpus=1) # specify the resources
@ray.remote # specify the resources
class Counter:
    def __init__(self):
        self.value = 20

    def increment(self, num):
        self.value = self.value + num
        return self.value

    def get_counter(self):
        return self.value

# Create an actor from this class.
counter = Counter.remote()

# Call the actor.
obj_ref = counter.increment.remote(10)
print(ray.get(obj_ref))



30


#### Multiple operations

In [52]:
# Create ten Counter actors.
counters = [Counter.remote() for _ in range(10)]

# Increment each Counter once and get the results. These tasks all happen in parallel.
results = ray.get([c.increment.remote(10) for c in counters])
print(results)


[30, 30, 30, 30, 30, 30, 30, 30, 30, 30]


In [53]:
# Increment the first Counter five times. These tasks are executed serially and share state.
results = ray.get([counters[0].increment.remote(20) for _ in range(10)])
print(results)

[50, 70, 90, 110, 130, 150, 170, 190, 210, 230]
